In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
data = pd.read_csv('diabetes_prediction_dataset.csv')
data

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0
...,...,...,...,...,...,...,...,...,...
99995,Female,80.0,0,0,No Info,27.32,6.2,90,0
99996,Female,2.0,0,0,No Info,17.37,6.5,100,0
99997,Male,66.0,0,0,former,27.83,5.7,155,0
99998,Female,24.0,0,0,never,35.42,4.0,100,0


In [3]:
data.isnull().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
smoking_history        0
bmi                    0
HbA1c_level            0
blood_glucose_level    0
diabetes               0
dtype: int64

In [4]:
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,Female,80.0,0,1,never,25.19,6.6,140,0
1,Female,54.0,0,0,No Info,27.32,6.6,80,0
2,Male,28.0,0,0,never,27.32,5.7,158,0
3,Female,36.0,0,0,current,23.45,5.0,155,0
4,Male,76.0,1,1,current,20.14,4.8,155,0


In [5]:
print(pd.unique(data['smoking_history'])) 

['never' 'No Info' 'current' 'former' 'ever' 'not current']


In [6]:
data['gender'] = data['gender'].map({'Female': 0, 'Male': 1})

In [7]:
data['smoking_history'] = data['smoking_history'].map({'never': 0, 'No Info': 1, 'current': 2, 'former': 3, 'ever': 4, 'not current': 5})

In [8]:
data.head()

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level,diabetes
0,0.0,80.0,0,1,0,25.19,6.6,140,0
1,0.0,54.0,0,0,1,27.32,6.6,80,0
2,1.0,28.0,0,0,0,27.32,5.7,158,0
3,0.0,36.0,0,0,2,23.45,5.0,155,0
4,1.0,76.0,1,1,2,20.14,4.8,155,0


In [24]:
data.isnull().sum()

gender                 18
age                     0
hypertension            0
heart_disease           0
smoking_history         0
bmi                     0
HbA1c_level             0
blood_glucose_level     0
diabetes                0
dtype: int64

In [26]:
data.dropna(inplace = True)

In [27]:
X=data.drop(['diabetes'], axis=1)

In [28]:
X

,gender,age,hypertension,heart_disease,smoking_history,bmi,HbA1c_level,blood_glucose_level
0,0.0,80.0,0,1,0,25.19,6.6,140
1,0.0,54.0,0,0,1,27.32,6.6,80
2,1.0,28.0,0,0,0,27.32,5.7,158
3,0.0,36.0,0,0,2,23.45,5.0,155
4,1.0,76.0,1,1,2,20.14,4.8,155
...,...,...,...,...,...,...,...,...
99995,0.0,80.0,0,0,1,27.32,6.2,90
99996,0.0,2.0,0,0,1,17.37,6.5,100
99997,1.0,66.0,0,0,3,27.83,5.7,155
99998,0.0,24.0,0,0,0,35.42,4.0,100


In [29]:
X.shape

(99982, 8)

In [30]:
y = data['diabetes']

In [31]:
y

0        0
1        0
2        0
3        0
4        0
        ..
99995    0
99996    0
99997    0
99998    0
99999    0
Name: diabetes, Length: 99982, dtype: int64

In [32]:
y.shape

(99982,)

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## RandomForest 

In [34]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_predictions = rf_model.predict(X_test)

## XGBoost

In [35]:
xgb_model = XGBClassifier()
xgb_model.fit(X_train, y_train)
xgb_predictions = xgb_model.predict(X_test)

## LSTM

In [36]:
X_train_reshaped = X_train.values.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test_reshaped = X_test.values.reshape((X_test.shape[0], 1, X_test.shape[1]))

lstm_model = Sequential()
lstm_model.add(LSTM(64, activation='relu', input_shape=(1, X_train.shape[1])))
lstm_model.add(Dense(1, activation='sigmoid'))

lstm_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
lstm_model.fit(X_train_reshaped, y_train, epochs=10, batch_size=32, validation_split=0.2)
lstm_predictions = lstm_model.predict(X_test_reshaped)

Epoch 1/10
2000/2000 [==============================] - 2s 736us/step - loss: 0.1667 - accuracy: 0.9432 - val_loss: 0.1512 - val_accuracy: 0.9432
Epoch 2/10
2000/2000 [==============================] - 1s 640us/step - loss: 0.1305 - accuracy: 0.9539 - val_loss: 0.1121 - val_accuracy: 0.9609
Epoch 3/10
2000/2000 [==============================] - 1s 636us/step - loss: 0.1248 - accuracy: 0.9541 - val_loss: 0.1141 - val_accuracy: 0.9592
Epoch 4/10
2000/2000 [==============================] - 1s 643us/step - loss: 0.1228 - accuracy: 0.9564 - val_loss: 0.1155 - val_accuracy: 0.9596
Epoch 5/10
2000/2000 [==============================] - 1s 629us/step - loss: 0.1207 - accuracy: 0.9575 - val_loss: 0.1153 - val_accuracy: 0.9589
Epoch 6/10
2000/2000 [==============================] - 1s 633us/step - loss: 0.1189 - accuracy: 0.9574 - val_loss: 0.1098 - val_accuracy: 0.9610
Epoch 7/10
2000/2000 [==============================] - 1s 630us/step - loss: 0.1193 - accuracy: 0.9585 - val_loss: 0.1191 -

### Combining all the 3 algorithms

In [37]:
ep = (rf_predictions + xgb_predictions + lstm_predictions.flatten()) / 3

### conversion to binary based on threshold

In [38]:
threshold = 0.5
final_predictions = (ep > threshold).astype(int)

In [39]:
accuracy = accuracy_score(y_test, final_predictions)
print(f"Ensemble Model Accuracy: {accuracy}")

Ensemble Model Accuracy: 0.9710456568485273
